In [ ]:
import numpy as np

In [ ]:
n_actions, n_states= 2,3
dtype = np.float32

In [ ]:
C = np.random.rand(n_actions, n_states, n_states).astype(dtype)
C

In [ ]:
pseudocount = 1e-3

In [ ]:
T = C + pseudocount
T

In [ ]:
norm = T.sum(2, keepdims=True)
norm

In [ ]:
norm[norm == 0] = 1
norm

In [ ]:
# C = np.random.rand(n_actions, n_states, n_states).astype(dtype)
T = C + pseudocount

norm = T.sum(2, keepdims=True)
# T /= (norm + alpha)
T /= norm
T

In [ ]:
T.sum(2, keepdims=True)

In [ ]:
np.shape(T) # action, state, state

### suppose new state --> new dimension = n_action, n_state+1, n_state+1

#### action, state = 1, 2

In [ ]:
alpha = 0.3
new_action = 1
new_state = 2
T[new_action, new_state, :] *= (1-alpha)
T.sum(2, keepdims=True)

In [ ]:
T = np.pad(T, ((0,0), (0,1), (0,1)), mode='constant', constant_values=pseudocount)
T

In [ ]:
T[new_action, new_state, -1] = alpha
T

#### For state t+1 -> state t: initialize based on random prob

In [ ]:
for action in range(n_actions):
    T[action, -1, :] = np.random.rand(len(T[action, -1, :]))
#     print(action)
T

#### Normalization

In [ ]:
# T = C + pseudocount

norm = T.sum(2, keepdims=True)
# T /= (norm + alpha)
T /= norm
T

In [ ]:
T.sum(2, keepdims=True)

In [ ]:
np.random.rand()

In [ ]:
T.sum(2, keepdims=True)

In [ ]:
# C = np.random.rand(n_actions, n_states, n_states).astype(dtype)
T = C + pseudocount

norm = T.sum(2, keepdims=True)
# T /= (norm + alpha)
T /= norm
T

In [ ]:
T.sum(2, keepdims=True)

In [ ]:
alpha = 0.1
T /= 

In [ ]:
T.sum(2, keepdims=True) + (alpha / (norm+alpha))

In [ ]:
# alpha = 10.0
norm = T.sum(2, keepdims=True)
T /= (norm + alpha)
T

In [ ]:
alpha / (norm+alpha)

In [ ]:
print(T.sum(2, keepdims=True))

In [ ]:
import numpy as np

# Initialize matrix size and pseudocount
n_actions = 5
n_states = 3
dtype = np.float32
pseudocount = 1e-3  # Small value to encourage new observations

# Initialize C matrix (transition counts or weights)
C = np.random.rand(n_actions, n_states, n_states).astype(dtype)

# Reserve space for new observations and normalize correctly
def update_transition_matrix(C, pseudocount, alpha=2.0):
    # Add pseudocount to encourage exploration
    T = C + pseudocount

    # Compute normalization term along the second dimension (across states)
    norm = T.sum(axis=2, keepdims=True) + alpha  # Ensure the second dimension sums to 1 with bias

    # Normalize probabilities by dividing each row-wise element by its norm
    T /= norm
    return T

# Example of expanding experiences: add new observations dynamically
def add_new_experience(C, new_state_action):
    # Expand the transition matrix to accommodate new states/actions
    new_C = np.pad(C, ((0, 0), (0, 1), (0, 1)), mode='constant', constant_values=0)

    # Update with new transition weights (e.g., based on new observation)
    action, from_state, to_state = new_state_action
    new_C[action, from_state, to_state] += 1.0  # Increment relevant transition count

    return new_C

# Update matrix with new experience
T = update_transition_matrix(C, pseudocount)
print("Initial Transition Matrix (Normalized):", T)

# Example: Add a new state-action experience and re-update matrix
new_experience = (2, 1, 3)  # action, from_state, to_state
C_expanded = add_new_experience(C, new_experience)
T_expanded = update_transition_matrix(C_expanded, pseudocount)
print("Expanded Transition Matrix (Normalized):", T_expanded)

# Check if second dimension sums to 1 for all (actions, states)
# print("Check if normalized (sums to 1):", np.allclose(T_expanded.sum(axis=1), 1.0))
print(T_expanded.sum(axis=2))


In [ ]:
import numpy as np
from scipy.stats import dirichlet, multinomial, norm, beta

class InfiniteHMM:
    def __init__(self, alpha=5.0, gamma=1.0, sigma_obs=1.0):
        self.alpha = alpha  # Concentration for transition matrix (Dirichlet Process)
        self.gamma = gamma  # Concentration for prior over states (HDP)
        self.sigma_obs = sigma_obs  # Observation noise

        self.states = []
        self.transitions = {}
        self.observations = []
        self.labels = []

    def generate_data(self, num_obs=50):
        """Generate synthetic data from a fixed HMM with infinite state space."""
        # True hidden states and corresponding observation mean values
        state_means = np.array([0.0, 2.5, 5.0])  # 3 potential states with distinct means
        state_sequence = np.random.choice(len(state_means), size=num_obs)  # State path

        self.observations = norm.rvs(loc=state_means[state_sequence], 
                                     scale=self.sigma_obs, size=num_obs)
        self.labels = state_sequence  # Store ground truth (only for comparison)

    def initialize_states(self):
        """Randomly initialize a sequence of states."""
        self.states = np.random.choice([0, 1, 2], size=len(self.observations)).tolist()

    def sample_new_state(self, obs, temp_states):
        """Gibbs sampling step: Sample a new state for a single observation."""
        unique_states = list(set(temp_states))  # Unique existing states

        # Calculate the likelihood for each existing state
        likelihoods = [
            norm.pdf(obs, loc=np.mean(self.observations[np.array(self.states) == s]),
                     scale=self.sigma_obs)
            for s in unique_states
        ]

        # Add the likelihood for a potential new state
        new_state_likelihood = norm.pdf(obs, loc=0.0, scale=self.sigma_obs)  # Prior mean 0
        likelihoods.append(new_state_likelihood)

        # Apply the gamma parameter to control the prior over new vs. existing states
        num_existing_states = len(unique_states)
        transition_probs = np.array(likelihoods) + self.gamma / (num_existing_states + 1)

        # Normalize to get probabilities
        probs = transition_probs / np.sum(transition_probs)

        # Sample a new state based on the computed probabilities
        return np.random.choice(len(probs), p=probs)


    def run_gibbs_sampling(self, num_iters=100):
        """Gibbs sampling over observations to infer state sequence."""
        for iteration in range(num_iters):
            for i, obs in enumerate(self.observations):
                # Temporarily store the current state to avoid dimension mismatch
                current_state = self.states[i]

                # Create a copy of states with the current state removed
                temp_states = self.states[:i] + self.states[i + 1:]

                # Sample a new state given the current observation
                new_state = self.sample_new_state(obs, temp_states)

                # Update the state in place
                self.states[i] = new_state

            if iteration % 10 == 0:
                print(f"Iteration {iteration}: States - {self.states}")

    def get_transition_matrix(self):
        """Construct a transition matrix from the inferred states."""
        # Get the number of unique states encountered
        unique_states = list(set(self.states))
        num_states = max(unique_states) + 1  # Ensure the matrix is large enough

        # Initialize the transition matrix with zeros
        transition_matrix = np.zeros((num_states, num_states))

        # Count transitions between inferred states
        for i in range(len(self.states) - 1):
            transition_matrix[self.states[i], self.states[i + 1]] += 1

        # Handle rows with no outgoing transitions to avoid division by zero
        with np.errstate(divide='ignore', invalid='ignore'):
            transition_matrix = (transition_matrix.T / transition_matrix.sum(axis=1)).T
            transition_matrix[np.isnan(transition_matrix)] = 0  # Replace NaNs with 0

        return transition_matrix


# Run the iHMM example
ihmm = InfiniteHMM(alpha=5.0, gamma=1.0, sigma_obs=1.0)



In [ ]:
ihmm.generate_data(num_obs=30)  # Generate synthetic observations


In [ ]:
ihmm.observations

In [ ]:
ihmm.labels

In [ ]:
ihmm.initialize_states()  # Initialize random state sequence
ihmm.states

In [ ]:
ihmm.run_gibbs_sampling(num_iters=500)  # Run Gibbs sampling


In [ ]:
print("\nInferred Transition Matrix:")
print(ihmm.get_transition_matrix())

In [ ]:
tm = ihmm.get_transition_matrix()
np.shape(tm)